In [1]:
import skimage
import numpy as np
import cv2

from sklearn import svm
from sklearn.externals import joblib 
from functions import show

from skimage import data, color, exposure
from skimage.feature import hog
import matplotlib.pyplot as plt

In [2]:
path = '/media/arjun/New Volume/Code/Research/Summer Stream Project/Content Based Image Retrieval/images/'

In [3]:
def extract(image,box=None):
    """Returns hog feature descriptor for an image.
    """
    if box is not None:
        image = image[box[0]:box[2],box[1]:box[3]]
        
    #image = color.rgb2gray(img)
    fd = hog(image, orientations=8, pixels_per_cell=(image.shape[1]/16, image.shape[0]/16),
                    cells_per_block=(1, 1), visualise=False, transform_sqrt=True)

#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

#     ax1.axis('off')
#     ax1.imshow(image, cmap=plt.cm.gray)
#     ax1.set_title('Input image')
#     ax1.set_adjustable('box-forced')

#     # Rescale histogram for better display
#     hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 0.02))

#     ax2.axis('off')
#     ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
#     ax2.set_title('Histogram of Oriented Gradients')
#     ax1.set_adjustable('box-forced')
#     plt.show()
    #print len(fd)
    return fd

In [4]:
bus,dinosaur,rose, elephant,horse= [],[],[],[],[]

for i in range(500,560):
    elephant.append(extract(cv2.imread(path+str(i)+'.jpg',0)))

for i in range(700,760):
    horse.append(extract(cv2.imread(path+str(i)+'.jpg',0)))

for i in range(600,660):
    rose.append(extract(cv2.imread(path+str(i)+'.jpg',0)))
    
for i in range(400,460):
    dinosaur.append(extract(cv2.imread(path+str(i)+'.jpg',0)))
    
for i in range(300,360):
    bus.append(extract(cv2.imread(path+str(i)+'.jpg',0)))

In [5]:
X = []
X.extend(elephant)
y = [4]*len(elephant)
X.extend(horse)
y.extend([3]*len(horse))
X.extend(rose)
y.extend([2]*len(rose))
X.extend(dinosaur)
y.extend([1]*len(dinosaur))
X.extend(bus)
y.extend([0]*len(bus))   
X = np.array(X)
y = np.array(y)
print 'Training'
print 'Feature Vectors:',len(X)
[i for i in range(len(X)) if len(X[i])!=2048]

Training
Feature Vectors: 300


[]

In [6]:
from sklearn.grid_search import GridSearchCV
print("Fitting the classifier to the training set")
param_grid = {'C': [0.01, 0.1, 1, 10, 100],'gamma':[0.01, 0.1, 1, 10, 100], 'kernel': ['rbf', 'linear']}
clf = GridSearchCV(svm.SVC(class_weight='balanced'), param_grid,n_jobs=2)
clf = clf.fit(X, y)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

# clf = svm.SVC(gamma=0.001, C=100,probability=True)
# clf.fit(X, y)

Fitting the classifier to the training set
Best estimator found by grid search:
SVC(C=10, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [7]:
Xtest = [extract(cv2.imread(path+str(i)+'.jpg',0)) for i in range(660,692)]
ytest = [2]*len(Xtest)

Xtest.extend([extract(cv2.imread(path+str(i)+'.jpg',0)) for i in range(560,585)])
ytest.extend([4]*len(range(560,585)))

Xtest.extend([extract(cv2.imread(path+str(i)+'.jpg',0)) for i in range(760,799)])
ytest.extend([3]*len(range(760,799)))

Xtest.extend([extract(cv2.imread(path+str(i)+'.jpg',0)) for i in range(360,390)])
ytest.extend([0]*len(range(360,390)))
Xtest.extend([extract(cv2.imread(path+str(i)+'.jpg',0)) for i in range(460,495)])
ytest.extend([1]*len(range(460,495)))
pred = clf.predict(Xtest)

In [8]:
from sklearn import metrics
print("Confusion matrix:\n%s" %metrics.confusion_matrix(ytest,pred))
print("Accuracy: %0.4f" % metrics.accuracy_score(ytest,pred))
print "{0}".format(metrics.classification_report(ytest,pred))

Confusion matrix:
[[29  0  0  0  1]
 [ 0 35  0  0  0]
 [ 0  0 28  1  3]
 [ 0  0  0 39  0]
 [ 0  0  0  4 21]]
Accuracy: 0.9441
             precision    recall  f1-score   support

          0       1.00      0.97      0.98        30
          1       1.00      1.00      1.00        35
          2       1.00      0.88      0.93        32
          3       0.89      1.00      0.94        39
          4       0.84      0.84      0.84        25

avg / total       0.95      0.94      0.94       161



In [9]:
clf.best_score_

0.92333333333333334